# Machine Learning for Fraud Detection.

##### **Notebook Author**: A.D. Téllez

You can find me on GitHub for collaborations.

## 1. Model Development

In [15]:
import pandas as pd

##### This data sets is under public domain.

In [16]:
card_df = pd.read_csv('card_transdata.csv')

#quick look to our data.
card_df.head() 

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0


#### Number of observations in our database.

In [17]:
print(card_df.shape[0],"observations.")

1000000 observations.


#### Looking for null values.

Sometimes we have null values in our data sets. Since they can compromise the performance of our model, we need to identify and treat them properly. Just deleting is not always the best solution.

In [18]:
card_df.isnull().sum()

distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64

#### Fraud incidences distribution on data set

In [19]:
fraud_df = card_df.value_counts(card_df['fraud']).rename(index = 'Number of cases').to_frame() #counts how many incidences belongs to each cases.  
fraud_df

,Number of cases
fraud,
0.0,912597
1.0,87403


#

### Equal outcome distribution 

There's a very unfair distribution between normal and fraudulent transactions. This can biasing our model.

#### 1. Separate the two classes 

In [20]:
# separating the normal transactions
legit = card_df[card_df.fraud == 0]

#look some statistical info about normal transactions
legit.distance_from_home.describe()

count    912597.000000
mean         22.832976
std          52.828655
min           0.004874
25%           3.828942
50%           9.673847
75%          24.158057
max        8777.136420
Name: distance_from_home, dtype: float64

In [21]:
# separating the normal transactions
fraud = card_df[card_df.fraud == 1]

#look some statistical info about fraudulent transactions
fraud.distance_from_home.describe()

count    87403.000000
mean        66.261876
std        134.391608
min          0.025847
25%          4.585729
50%         15.454219
75%        101.110104
max      10632.723672
Name: distance_from_home, dtype: float64

**After that, we see fraudulent transactions has a correlation with the distance.** 

We can extend this exploratory analysis for other attributes on the dataframe. We selected just one in order to eliminate distracting values.   

#### Pick an equal distribution of each class

In [22]:
legit_sample = legit.sample(n = 87403) #n is the number of random observations to make a sample set. 

#Also n = number of fraudulent transactions

In [23]:
# We join the now balanced 2 datasets

# legit_sample -> the reduced dataset of legit transactions
# fraud -> dataset with fraudulent transactions

sample = pd.concat( [legit_sample, fraud] , axis = 0) # axis = 0 concatenate rows 

In [26]:
sample.value_counts(sample['fraud'].rename(index = 'Number of cases'))

Number of cases
0.0    87403
1.0    87403
dtype: int64

#

#### Standardization

This process prepares our data in such a way that the mean is zero. This sets our values within the range [-1,1] preventing extreme values from weighting more in our model.  

In [57]:
# allows us to standarsize our data.
from sklearn import preprocessing 

#### Understanding the process

When setting our model, we need some input data called independent variables. But we also need to define what would be the expected outcome. The variable we would like to predict, categorize or cluster is called the dependent variable.  

In [58]:
transform = preprocessing.StandardScaler()  # performs the standardization.

X = transform.fit_transform(card_df) #Independent variables on the x-axis.
X

array([[ 0.47788202, -0.18284892,  0.04349141, ..., -0.33445812,
        -1.36442519, -0.30947363],
       [-0.24160679, -0.18809398, -0.18930045, ..., -0.33445812,
        -1.36442519, -0.30947363],
       [-0.3293694 , -0.16373307, -0.49881185, ..., -0.33445812,
         0.73290937, -0.30947363],
       ...,
       [-0.36264968, -0.13790278, -0.57369398, ..., -0.33445812,
         0.73290937, -0.30947363],
       [-0.34209827, -0.1855234 , -0.48162807, ..., -0.33445812,
         0.73290937, -0.30947363],
       [ 0.48140344, -0.18257921, -0.51338354, ..., -0.33445812,
         0.73290937, -0.30947363]])

In [59]:
Y = card_df['fraud'].to_numpy()
Y

array([0., 0., 0., ..., 0., 0., 0.])

### Split the data -- train vs test

In [60]:
# Allows us to split our data into training and testing data
from sklearn.model_selection import train_test_split

# Allows us to test parameters of classification algorithms and find the best one
from sklearn.model_selection import GridSearchCV

In [61]:
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size = 0.999, random_state = 2)

In [62]:
X_train.shape

(1000, 8)

In [43]:
# Logistic Regression classification algorithm
from sklearn.linear_model import LogisticRegression

In [44]:
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

In [45]:
parameters ={"C":[0.01,0.1,1],'penalty':['l2'], 'solver':['lbfgs']}# l1 lasso l2 ridge
lr=LogisticRegression()

logreg_cv = GridSearchCV(estimator = lr, param_grid = parameters, cv=10)
logreg_cv.fit(X_train,Y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1], 'penalty': ['l2'],
                         'solver': ['lbfgs']})

In [46]:
print("accuracy :",logreg_cv.best_score_)

accuracy : 0.9999971428571428


In [63]:
# K Nearest Neighbors classification algorithm
from sklearn.neighbors import KNeighborsClassifier

In [64]:
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

KNN = KNeighborsClassifier()

In [65]:
knn_cv = GridSearchCV (KNN, parameters, cv = 10)
knn_cv.fit(X_train, Y_train)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'p': [1, 2]})

In [66]:
print("accuracy :",knn_cv.best_score_)

accuracy : 1.0


In [71]:
Y_train

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0.